# **\[Essays\]** 강화학습(2) – 4x4 grid

신록예찬  
2023-08-23

## Game2: 4 $\times$ 4 그리드

## imports

In [1]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import IPython

## 예비학습: 시각화

In [2]:
def show(states):
    fig = plt.Figure()
    ax = fig.subplots()
    ax.matshow(np.zeros([4,4]), cmap='bwr',alpha=0.0)
    sc = ax.scatter(0, 0, color='red', s=500)  
    ax.text(0, 0, 'start', ha='center', va='center')
    ax.text(3, 3, 'end', ha='center', va='center')
    # Adding grid lines to the plot
    ax.set_xticks(np.arange(-.5, 4, 1), minor=True)
    ax.set_yticks(np.arange(-.5, 4, 1), minor=True)
    ax.grid(which='minor', color='black', linestyle='-', linewidth=2)
    def update(t):
        sc.set_offsets(states[t])
    ani = FuncAnimation(fig,update,frames=len(states))
    display(IPython.display.HTML(ani.to_jshtml()))

In [3]:
states = [[0,0],[0,1],[1,1],[1,2],[1,3]]

In [4]:
show(states)

## Env 클래스 구현

`-` GridWorld: 강화학습에서 많이 사용되는 기본적인 시뮬레이션 환경

1.  **State**: 각 격자 셀이 하나의 상태이며, 에이전트는 이러한 상태 중
    하나에 있을 수 있음.
2.  **Action**: 에이전트는 상태에서 다른 상태로 이동하기 위해 상, 하,
    좌, 우로 이동하는 행동을 할 수 있음.
3.  **Reward**: 에이전트가 특정 행동을 취할 때 환경에서 보상이 주어짐.
4.  **Terminal State**: 일반적으로 하나 또는 그 이상의 종료 상태가
    있으며, 에이전트가 이 상태에 도달하면 에피소드가 종료됨.

In [5]:
class GridWorld:
    def __init__(self):
        self.reset()
        self._action_to_direction = {
            0: np.array([1, 0]), # x+ 
            1: np.array([0, 1]), # y+
            2: np.array([-1, 0]), # x-  
            3: np.array([0, -1]), # y-
        }
        self.state_space = gym.spaces.MultiDiscrete([4, 4])
        self.action_space = gym.spaces.Discrete(4)
    def reset(self):
        self.agent_action = None
        self.agent_state = np.array([0, 0])
        return self.agent_state
    def step(self,action):
        direction = self._action_to_direction[action]
        self.agent_state = self.agent_state + direction
        # 목표지점에 도달 
        if np.array_equal(np.array([3,3]), self.agent_state):
            reward = 100
            terminated = True
        else:
            reward = -1
            terminated = False
        # 4*4밖에 있을 경우 
        if self.agent_state not in self.state_space:
            reward = -10
            terminated = True
            self.agent_state = self.agent_state - 1/2*direction
        return self.agent_state, reward, terminated

In [6]:
env = GridWorld()
env.reset()
states = []
rewards = [] 
terminations = [] 
for t in range(500):
    action = env.action_space.sample()
    state, reward, terminated = env.step(action)
    states.append(state)
    rewards.append(reward)
    terminations.append(terminated)
    if terminated: 
        break 

In [7]:
[np.array([0,0])]+states

In [8]:
show([np.array([0,0])]+states)

## Agent1 클래스 구현 + Run

`-` 첫번째 시도

In [9]:
# learn 추가
class Agent1:
    def __init__(self,env):
        self.action_space = env.action_space
        self.state_space = env.state_space
        self.n_experiences = 0
        self.n_episode = 0  
        
        ## episode-wise info 
        self.scores = [] 
        self.playtimes = [] 

        ## time-wise info
        self.current_state = None 
        self.action = None
        self.reward = None        
        self.next_state = None 
        self.socre = 0
        
        ## ReplayBuffer
        self.actions = []
        self.rewards = []
        self.current_states = []
        self.next_states = [] 
        self.terminations = []
        
    def act(self):
        self.action = self.action_space.sample()
        
    def save_experience(self): 
        self.actions.append(self.action)
        self.current_states.append(self.current_state)
        self.next_states.append(self.next_state)
        self.rewards.append(self.reward)
        self.terminations.append(self.terminated)
        self.n_experiences += 1
        self.score += self.reward

    def learn(self):
        pass

In [10]:
env = GridWorld()
agent = Agent1(env)
for _ in range(20):
    ### 1. 본질적인 코드
    agent.current_state = env.reset() 
    agent.terminated = False
    agent.score = 0 
    for t in range(50):
        # step1: agent >> env 
        agent.act()
        env.agent_action = agent.action
        # step2: env << agent 
        agent.next_state, agent.reward, agent.terminated = env.step(env.agent_action)
        agent.save_experience()
        # step3: 학습
        agent.learn()
        # step4: state update 
        agent.current_state = agent.next_state 
        # step5: 종료조건 체크
        if agent.terminated: break 
    agent.scores.append(agent.score) 
    agent.playtimes.append(t+1)
    agent.n_episode = agent.n_episode + 1 
    ## 2. 비본질적 코드
    print(
        f'Episode {agent.n_episode}\t'
        f'Score: {agent.scores[-1]}\t'
        f'Playtime: {agent.playtimes[-1]}'
    )

Episode 1   Score: -16  Playtime: 7
Episode 2   Score: -12  Playtime: 3
Episode 3   Score: -10  Playtime: 1
Episode 4   Score: -11  Playtime: 2
Episode 5   Score: -10  Playtime: 1
Episode 6   Score: -11  Playtime: 2
Episode 7   Score: -10  Playtime: 1
Episode 8   Score: -19  Playtime: 10
Episode 9   Score: -10  Playtime: 1
Episode 10  Score: -12  Playtime: 3
Episode 11  Score: -22  Playtime: 13
Episode 12  Score: -10  Playtime: 1
Episode 13  Score: -10  Playtime: 1
Episode 14  Score: -12  Playtime: 3
Episode 15  Score: -11  Playtime: 2
Episode 16  Score: -10  Playtime: 1
Episode 17  Score: -16  Playtime: 7
Episode 18  Score: -10  Playtime: 1
Episode 19  Score: -20  Playtime: 11
Episode 20  Score: -10  Playtime: 1

`-` 어떻게 학습을 할까? 즉 어떻게 “환경의 이해 $\to$ 행동의 결정” 의
과정을 수행할까?

1.  어떠한 상태에서, 어떠한 행동을 했을때, 어떠한 보상과 어떠한
    다음상태를 받았는지 기록하자.
2.  1을 바탕으로 다음행동을 어떻게 할지 판단하자.

In [11]:
states = [np.array([0,0])]+agent.next_states[-5:-1]
states

In [12]:
show(states)

## 환경의 이해 (1차원적 이해)

`-` 무작위로 10000판을 진행하여 보자.

In [13]:
env = GridWorld()
agent = Agent1(env)
for _ in range(10000):
    ### 1. 본질적인 코드
    agent.current_state = env.reset() 
    agent.terminated = False
    agent.score = 0 
    for t in range(50):
        # step1: agent >> env 
        agent.act()
        env.agent_action = agent.action
        # step2: agent << env
        agent.next_state, agent.reward, agent.terminated = env.step(env.agent_action)
        agent.save_experience()
        # step3: 학습
        agent.learn()
        # step4: state update 
        agent.current_state = agent.next_state 
        # step5: 종료조건 체크
        if agent.terminated: break 
    agent.scores.append(agent.score) 
    agent.playtimes.append(t+1)
    agent.n_episode = agent.n_episode + 1 

In [14]:
agent.n_experiences

`-` 데이터관찰

In [15]:
agent.current_states[0], agent.actions[0], agent.rewards[0]

In [16]:
agent.current_states[1], agent.actions[1], agent.rewards[1]

In [17]:
agent.current_states[2], agent.actions[2], agent.rewards[2]

In [18]:
agent.current_states[3], agent.actions[3], agent.rewards[3]

`-` 환경을 이해하기 위한 기록 (1)

In [52]:
q = np.zeros([4,4,4])
count = np.zeros([4,4,4])
for i in range(agent.n_experiences):
    x,y = agent.current_states[i]
    a = agent.actions[i]
    q[x,y,a] = q[x,y,a] + agent.rewards[i]
    count[x,y,a] = count[x,y,a] + 1 

In [53]:
count[count==0] = 0.1
count

In [54]:
q = (q/count).round(2)
q

In [55]:
for i in range(4):
    print(f"action = {i}\n"
          f"action-value function =\n {q[:,:,i]}\n"
          )

action = 0
action-value function =
 [[ -1.  -1.  -1.  -1.]
 [ -1.  -1.  -1.  -1.]
 [ -1.  -1.  -1. 100.]
 [-10. -10. -10.   0.]]

action = 1
action-value function =
 [[ -1.  -1.  -1. -10.]
 [ -1.  -1.  -1. -10.]
 [ -1.  -1.  -1. -10.]
 [ -1.  -1. 100.   0.]]

action = 2
action-value function =
 [[-10. -10. -10. -10.]
 [ -1.  -1.  -1.  -1.]
 [ -1.  -1.  -1.  -1.]
 [ -1.  -1.  -1.   0.]]

action = 3
action-value function =
 [[-10.  -1.  -1.  -1.]
 [-10.  -1.  -1.  -1.]
 [-10.  -1.  -1.  -1.]
 [-10.  -1.  -1.   0.]]


`-` 환경을 이해하기 위한 기록 (2) – 이렇게하면 count를 따로 기록할 필요
없음

In [56]:
q = np.zeros([4,4,4])
for i in range(agent.n_experiences):
    x,y = agent.current_states[i]
    a = agent.actions[i]
    q_estimated = q[x,y,a] # 풀이한 답
    q_realistic = agent.rewards[i] # 실제 답
    diff = q_realistic - q_estimated # 실제답과 풀이한값의 차이 = 오차피드백값이라고 하자
    q[x,y,a] = q_estimated + 0.05 * diff ## 새로운답 = 원래답 + 오차피드백 * 학습률

In [24]:
for i in range(4):
    print(
        f"action = {i}\n"
        f"action-value function =\n {q[:,:,i].round(1)}\n"
    )

action = 0
action-value function =
 [[ -1.   -1.   -1.   -1. ]
 [ -1.   -1.   -1.   -1. ]
 [ -1.   -1.   -1.   98.8]
 [-10.  -10.   -9.8   0. ]]

action = 1
action-value function =
 [[ -1.  -1.  -1. -10.]
 [ -1.  -1.  -1. -10.]
 [ -1.  -1.  -1. -10.]
 [ -1.  -1.  98.   0.]]

action = 2
action-value function =
 [[-10. -10. -10. -10.]
 [ -1.  -1.  -1.  -1.]
 [ -1.  -1.  -1.  -1.]
 [ -1.  -1.  -1.   0.]]

action = 3
action-value function =
 [[-10.  -1.  -1.  -1.]
 [-10.  -1.  -1.  -1.]
 [-10.  -1.  -1.  -1.]
 [-10.  -1.  -1.   0.]]


## 환경의 깊은 이해 (좀 더 고차원적인 이해)

`-` action=1 일때 각 state의 가치 (=기대보상)

In [51]:
q[:,:,1]

`-` 분석1

In [26]:
q[3,2][1] 

-   상태 (3,2)에서 행동 1을 하게 되면 100의 보상을 얻으므로 기대보상값은
    100근처 -\> 합리적임

`-` 분석2

In [27]:
q[3,1][1] 

-   상태 (3,1)에서 행동1을 하게 되면 -1의 보상을 얻으므로 기대보상값은
    -1 근처 -\> 합리적일까?

`-` 비판: 분석2는 합리적인듯 하지만 data를 분석한뒤는 그다지 합리적이지
못함

`-` 상황상상

-   빈 종이를 줌
-   빈 종이에는 0 또는 1을 쓸 수 있음 (action = 0 or 1)
-   0을 쓸 때와 1을 쓸 때의 보상은 다름
-   그런데 무수히 많은 데이터를 분석한 결과 0을 쓰면 0원을 보상으로
    주고, 1을 쓰면 10만원을 보상으로 준다는 것을 “알게 되었음”
-   빈 종이의 가치는 5만원인가? 아니면 10만원인가? –\> 10만원 아니야?

`-` 직관: 생각해보니 현재 $s=(3,1)$, $a=1$에서 추정된(esimated) 값은
`q[3,1][1]`이지만[1], 현실적으로는 “실제보상(-1)과 잠재적보상(100)”을
동시에 고려해야하는게 합리적인듯

[1] 즉 잠재적 보상은 고려되어있지 않음

In [28]:
q_esimated = q[3,1][1]
q_esimated

In [29]:
q_realistic = (-1) + 0.99 * (100) 
q_realistic

-   여기에서 0.99 는 “미래에 받을 보상이 현재에 비해 얼마나 중요한지를
    결정하는 가중치” 이다.
-   1에 가까울수록 미래에 받을 보상을 매우 중시한다는 의미 (즉 빈종이 =
    십만원 으로 생각한다는 의미)

`-` 즉 $q(s,a)$는 모든 $s,a$에 대하여

$$q(s,a) \approx \text{reward}(s,a) + 0.99\times \max_{a} q(s',a)$$

가 성립한다면 $q(s,a)$ 타당하게 추정된 것이라 볼 수 있다. 좀 더
정확하게는

$$q(s,a) \approx \begin{cases} \text{reward}(s,a) & \text{terminated}  \\ \text{reward}(s,a)+ 0.99\times \max_{a} q(s',a) & \text{not terminated} \end{cases} $$

In [30]:
q = np.zeros([4,4,4])
for i in range(agent.n_experiences):
    x,y = agent.current_states[i]
    xx,yy = agent.next_states[i]
    a = agent.actions[i]
    q_estimated = q[x,y,a]
    if agent.terminations[i]:
        q_realistic = agent.rewards[i] 
    else:
        q_future = q[xx,yy,:].max()
        q_realistic = agent.rewards[i] + 0.99*q_future # 이걸 관측했다고 치는거임
    diff = q_realistic - q_estimated 
    q[x,y,a] = q_estimated + 0.1 * diff

In [31]:
for i in range(4):
    print(
        f"action = {i}\n"
        f"action-value function =\n {q[:,:,i].round(1)}\n"
    )

action = 0
action-value function =
 [[ 90.2  92.1  94.   95.8]
 [ 92.1  94.   96.   98. ]
 [ 93.8  95.9  98.  100. ]
 [-10.  -10.  -10.    0. ]]

action = 1
action-value function =
 [[ 90.2  92.1  93.8 -10. ]
 [ 92.1  94.   96.  -10. ]
 [ 94.   96.   98.  -10. ]
 [ 95.8  97.9 100.    0. ]]

action = 2
action-value function =
 [[-10.  -10.  -10.  -10. ]
 [ 88.3  90.2  92.1  93.2]
 [ 90.2  92.1  94.   95.4]
 [ 92.1  94.   96.    0. ]]

action = 3
action-value function =
 [[-10.   88.3  90.2  92.1]
 [-10.   90.2  92.1  94. ]
 [-10.   92.1  94.   95.9]
 [-10.   93.6  95.2   0. ]]


## 행동 전략 수립

`-` 상태 (0,0)에 있다고 가정해보자.

In [32]:
q[0,0,:]

-   행동 0 혹은 1을 하는게 유리함.

`-` 상태 (2,3)에 있다고 가정해보자.

In [33]:
q[2,3,:]

-   행동 0을 하는게 유리함.

`-` 상태 (3,2)에 있다고 가정해보자.

In [34]:
q[3,2,:] 

-   행동 1을 하는게 유리함.

`-` 각 상태에서 최적은 action은 아래와 같다.

In [35]:
q[0,0,:].argmax()

In [36]:
q[2,3,:].argmax()

In [37]:
q[3,2,:].argmax()

`-` 전략(=정책)을 정리해보자.

In [38]:
policy = np.array(['?????']*16).reshape(4,4)
policy

In [39]:
directions = {0: 'down', 1: 'right', 2:'up', 3:'left'}

In [40]:
for i in range(4):
    for j in range(4):
        policy[i,j] = directions[q[i,j,:].argmax()]
policy

`-` 요약: 값이 큰 쪽으로 이동

In [41]:
q.max(axis=-1)

## Agent2 클래스 구현 + Run

In [42]:
# learn 추가
class Agent2(Agent1):
    def __init__(self,env):
        super().__init__(env)
        self.q = np.zeros([4,4,4])
    def act(self):
        if self.n_experiences < 3000: 
            self.action = self.action_space.sample()
        else: 
            x,y = self.current_state
            self.action = self.q[x,y,:].argmax()
    def learn(self): # make q
        x,y = self.current_state
        xx,yy = self.next_state
        a = self.action
        q_estimated = self.q[x,y,a]
        if agent.terminated:
            q_realistic = self.reward
        else:
            q_future = q[xx,yy,:].max()
            q_realistic = self.reward + 0.99*q_future
        # q_observed 와 q_estimated를 점점 비슷하게 만들어주는 역할
        diff = q_realistic - q_estimated
        self.q[x,y,a] = q_estimated + 0.1 * diff 

In [43]:
env = GridWorld()
agent = Agent2(env)
for _ in range(2000):
    ### 1. 본질적인 코드
    agent.current_state = env.reset() 
    agent.terminated = False
    agent.score = 0 
    for t in range(50):
        # step1: agent >> env 
        agent.act()
        env.agent_action = agent.action
        # step2: env << agent 
        agent.next_state, agent.reward, agent.terminated = env.step(env.agent_action)
        agent.save_experience()
        # step3: 데이터저장 및 학습
        agent.learn()        
        # step4: 다음 iteration 준비 + 종료조건체크
        agent.current_state = agent.next_state 
        if agent.terminated: break 
    agent.scores.append(agent.score) 
    agent.playtimes.append(t+1)
    agent.n_episode = agent.n_episode + 1 
    ## 2. 비본질적 코드
    if (agent.n_episode % 100) == 0:
        print(
            f'Episode {agent.n_episode}\t'
            f'Score: {np.mean(agent.scores[-100:]) : .2f}\t'
            f'Playtime: {np.mean(agent.playtimes[-100:]) : .2f}\t'
            f'n_experiences: {agent.n_experiences}'
        )

Episode 100 Score: -8.30    Playtime:  3.70 n_experiences: 370
Episode 200 Score: -10.66   Playtime:  2.76 n_experiences: 646
Episode 300 Score: -12.63   Playtime:  3.63 n_experiences: 1009
Episode 400 Score: -6.53    Playtime:  3.03 n_experiences: 1312
Episode 500 Score: -10.20   Playtime:  3.40 n_experiences: 1652
Episode 600 Score: -11.89   Playtime:  2.89 n_experiences: 1941
Episode 700 Score: -12.92   Playtime:  3.92 n_experiences: 2333
Episode 800 Score: -10.03   Playtime:  3.23 n_experiences: 2656
Episode 900 Score: -10.12   Playtime:  3.32 n_experiences: 2988
Episode 1000    Score: -48.86   Playtime:  48.59    n_experiences: 7847
Episode 1100    Score: -50.00   Playtime:  50.00    n_experiences: 12847
Episode 1200    Score: -50.00   Playtime:  50.00    n_experiences: 17847
Episode 1300    Score: -50.00   Playtime:  50.00    n_experiences: 22847
Episode 1400    Score: -50.00   Playtime:  50.00    n_experiences: 27847
Episode 1500    Score: -50.00   Playtime:  50.00    n_experien

In [44]:
states = [np.array([0,0])] + agent.next_states[-agent.playtimes[-1]:]
show(states)

## Agent3 클래스 구현 + Run

In [45]:
class Agent3(Agent2):
    def __init__(self,env):
        super().__init__(env)
        self.eps = 0
    def act(self):
        if np.random.rand() < self.eps:
            self.action = self.action_space.sample()
        else: 
            x,y = self.current_state
            self.action = self.q[x,y,:].argmax()

In [46]:
env = GridWorld()
agent = Agent3(env)
agent.eps = 1 
for _ in range(5000):
    ### 1. 본질적인 코드
    agent.current_state = env.reset() 
    agent.terminated = False
    agent.score = 0 
    for t in range(50):
        # step1: agent >> env 
        agent.act()
        env.agent_action = agent.action
        # step2: env << agent 
        agent.next_state, agent.reward, agent.terminated = env.step(env.agent_action)
        agent.save_experience()
        # step3: 데이터저장 및 학습
        agent.learn()        
        # step4: 다음 iteration 준비 + 종료조건체크
        agent.current_state = agent.next_state 
        if agent.terminated: break 
    agent.scores.append(agent.score) 
    agent.playtimes.append(t+1)
    agent.n_episode = agent.n_episode + 1 
    agent.eps = agent.eps * 0.999
    ## 2. 비본질적 코드
    if (agent.n_episode % 1000) == 0:
        print(
            f'Episode {agent.n_episode}\t'
            f'Score: {np.mean(agent.scores[-100:]) : .2f}\t'
            f'Playtime: {np.mean(agent.playtimes[-100:]) : .2f}\t'
            f'n_experiences: {agent.eps}'
        )

Episode 1000    Score:  35.77   Playtime:  5.83 n_experiences: 0.3676954247709635
Episode 2000    Score:  78.15   Playtime:  6.35 n_experiences: 0.1351999253974994
Episode 3000    Score:  85.34   Playtime:  5.76 n_experiences: 0.04971239399803625
Episode 4000    Score:  92.77   Playtime:  6.03 n_experiences: 0.018279019827489446
Episode 5000    Score:  91.76   Playtime:  5.94 n_experiences: 0.006721111959865607

In [47]:
states = [np.array([0,0])] + agent.next_states[-agent.playtimes[-1]:]
show(states)

## 최종 Agent 클래스 구현 + Run

In [48]:
# learn 추가
class Agent(Agent3):
    # agent >> env 
    def __rshift__(self,env):
        self.act()
        env.agent_action = self.action
    # agent << env 
    def __lshift__(self,env): 
        self.next_state, self.reward, self.terminated = env.step(env.agent_action)
        self.save_experience()

In [49]:
env = GridWorld()
agent = Agent(env)
agent.eps = 1 
for _ in range(5000):
    ### 1. 본질적인 코드
    agent.current_state = env.reset() 
    agent.terminated = False
    agent.score = 0 
    for t in range(50):
        ## step1 
        agent >> env 
        ## step2 
        agent << env 
        ## step3 
        agent.learn()    
        ## step4 
        agent.current_state = agent.next_state 
        ## step5 
        if agent.terminated: break 
    agent.scores.append(agent.score) 
    agent.playtimes.append(t+1)
    agent.n_episode = agent.n_episode + 1 
    agent.eps = agent.eps*0.999
    ## 2. 비본질적 코드
    if (agent.n_episode % 1000) == 0:
        print(
            f'Episode {agent.n_episode}\t'
            f'Score: {np.mean(agent.scores[-100:]) : .2f}\t'
            f'Playtime: {np.mean(agent.playtimes[-100:]) : .2f}\t'
            f'n_eps: {agent.eps}'
        )

Episode 1000    Score:  39.50   Playtime:  6.50 n_eps: 0.3676954247709635
Episode 2000    Score:  71.99   Playtime:  5.91 n_eps: 0.1351999253974994
Episode 3000    Score:  92.76   Playtime:  6.04 n_eps: 0.04971239399803625
Episode 4000    Score:  92.82   Playtime:  5.98 n_eps: 0.018279019827489446
Episode 5000    Score:  93.86   Playtime:  6.04 n_eps: 0.006721111959865607

In [50]:
states = [np.array([0,0])] + agent.next_states[-agent.playtimes[-1]:]
show(states)